# Wedge Task 3
___
The third task of this project involves building several summary text files and populate a relational database using these files.
### Connect to GBQ
___

In [1]:
from google.cloud import bigquery
import gc
import pandas as pd
import sqlite3

In [2]:
client = bigquery.Client()

### Build Summary Tables
___

In [3]:
def execute_query_to_file(query, filename):
    try:
        # Execute the query
        query_job = client.query(query)
        # Convert the query results to a list of dictionaries
        results = [dict(row) for row in query_job]
        # Convert the results into a pandas DataFrame
        results_df = pd.DataFrame(results)
        # Save to a txt file in the data folder
        results_df.to_csv(filename, index=False, sep=';')
        # Memory management
        del results, results_df
        gc.collect()
        # Success message
        print("File saved successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

##### First File
___
Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day determine the total spend in the store, the number of transactions, and a count of the number of items

In [4]:
query1 = """
SELECT EXTRACT(DATE FROM CAST(datetime AS DATETIME)) AS date,
       EXTRACT(HOUR FROM CAST(datetime AS DATETIME)) AS hour,
       SUM(total) AS spend,
       COUNT(DISTINCT CONCAT(
                            CAST(EXTRACT(DATE FROM CAST(datetime AS DATETIME)) AS STRING),
                            CAST(register_no AS STRING),
                            CAST(emp_no AS STRING),
                            CAST(trans_no AS STRING)
                            )
              ) AS transactions,
       SUM(CASE
              WHEN trans_status IN ('V', 'R') THEN -1
              ELSE 1
              END
              ) AS items
FROM `wedge-to-the-cloud.wedge_to_the_dataset.transactions`
WHERE department NOT IN (-1,0,15,99)
       AND (trans_status IN (' ', '', 'V', 'R') OR trans_status IS NULL)
GROUP BY date, hour
ORDER BY date, hour
"""

filename1 = 'data/SummaryTables/sales_by_date_by_hour.txt'

execute_query_to_file(query1, filename1)

File saved successfully.


##### Second File
___
Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items.

In [5]:
query2 = """
SELECT card_no,
       EXTRACT(YEAR FROM CAST(datetime AS DATETIME)) AS year,
       EXTRACT(MONTH FROM CAST(datetime AS DATETIME)) AS month,
       SUM(total) AS spend,
       COUNT(DISTINCT CONCAT(
                            CAST(EXTRACT(DATE FROM CAST(datetime AS DATETIME)) AS STRING),
                            CAST(register_no AS STRING),
                            CAST(emp_no AS STRING),
                            CAST(trans_no AS STRING)
                            )
              ) AS transactions,
       SUM(CASE
              WHEN trans_status IN ('V', 'R') THEN -1
              ELSE 1
              END
              ) AS items
FROM `wedge-to-the-cloud.wedge_to_the_dataset.transactions`
WHERE department NOT IN (-1,0,15,99)
       AND (trans_status IN (' ', '', 'V', 'R') OR trans_status IS NULL)
GROUP BY card_no, year, month
ORDER BY card_no, year, month
"""

filename2 = 'data/SummaryTables/sales_by_owner_by_year_by_month.txt'

execute_query_to_file(query2, filename2)

File saved successfully.


##### Third File
___
Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [6]:
query3 = """
SELECT upc,
       description,
       department,
       EXTRACT(YEAR FROM CAST(datetime AS DATETIME)) AS year,
       EXTRACT(MONTH FROM CAST(datetime AS DATETIME)) AS month,
       SUM(total) AS spend,
       COUNT(DISTINCT CONCAT(
                            CAST(EXTRACT(DATE FROM CAST(datetime AS DATETIME)) AS STRING),
                            CAST(register_no AS STRING),
                            CAST(emp_no AS STRING),
                            CAST(trans_no AS STRING)
                            )
              ) AS transactions,
       SUM(CASE
              WHEN trans_status IN ('V', 'R') THEN -1
              ELSE 1
              END
              ) AS items
FROM `wedge-to-the-cloud.wedge_to_the_dataset.transactions`
WHERE department NOT IN (-1,0,15,99)
       AND (trans_status IN (' ', '', 'V', 'R') OR trans_status IS NULL)
GROUP BY upc, description, department, year, month
ORDER BY upc, description, department, year, month
"""

filename3 = 'data/SummaryTables/sales_by_product_description_by_year_by_month.txt'

execute_query_to_file(query3, filename3)

File saved successfully.


### Build Database
___

In [7]:
def write_file_to_database(filename):
    try:
        # Create a connection to the database
        connection = sqlite3.connect('data/wedge-to-the-database.db')
        # Read the file
        df = pd.read_csv(filename, sep=';')
        # Get the table name from the file name
        tablename = filename.split('/')[-1].split('.')[0]
        # Put the dataframe into the sql database file
        df.to_sql(tablename, connection, if_exists='replace', index=False)
        # Memory management
        del df, tablename
        gc.collect()
        # Success flag
        print("Database table saved successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the connection
        connection.close()

##### First File
___

In [8]:
write_file_to_database(filename1)

Database table saved successfully.


##### Second File
___

In [9]:
write_file_to_database(filename2)

Database table saved successfully.


##### Third File
___

In [10]:
write_file_to_database(filename3)

/var/folders/71/xfbtbhlj7376x9dz94q5nmbh0000gn/T/ipykernel_32559/1796808687.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, sep=';')


Database table saved successfully.
